In [1]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os 
import csv

Using matplotlib backend: <object object at 0x0000016F3D937590>


In [153]:
def createFissMatr(fissions, chis):
    fissMatr = np.zeros((len(fissions),len(fissions)))
    for i in range(0, len(fissMatr)):
        for j in range(0,len(fissMatr[0])):
            fissMatr[i][j] = fissions[j]*chis[i]
    return fissMatr

In [154]:
def readCSV(filename):
    
    absorptions = []
    fissions = []
    chis = []
    SctData = []
    
    with open(filename, 'r') as file:
        
        csvreader = csv.reader(file)
        next(csvreader)#Title for Data
        next(csvreader)#Titles of Cross-Sections
        i=0
        
        
        for row in csvreader:
            #print(row)
            try:
                absorptions.append(float(row[1]))
                fissions.append(float(row[2]))
                chis.append(float(row[3]))
                i+=1
            except:
                
                break
        
        next(csvreader) #skips straight to next data
        
            
        length = len(absorptions)+1
        #print(length)
        
        for row in csvreader:
            #print(row)
            
            SctData.append([float(number) for number in row[1:length]])
            
    #print(absorptions)
    #print(fissions)
    #print(chis)
    #print(SctData)
    
    absMatr = np.diag(absorptions)
    
    fissMatr = createFissMatr(fissions,chis)
    #print(fissMatr)
    
    inSctMatr = []
    outSctMatr = []
    
    data = {}    
    return data

In [156]:
readCSV("InputDataCSVGroup8.csv")

[[0.00469938 0.00196392 0.00038577 0.00234969 0.0077154  0.00778554
  0.03145779 0.07508487]
 [0.0055007  0.0022988  0.00045155 0.00275035 0.009031   0.0091131
  0.03682185 0.08788805]
 [0.00319992 0.00133728 0.00026268 0.00159996 0.0052536  0.00530136
  0.02142036 0.05112708]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]


{}